# Slow Fusion

This script takes all frames at once and passes them through a cnn.

In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import initializers
import time
import numpy as np

## 0. Define Region

First, define a region to train a model for.

In [2]:
REGION = 0
REGIONS = ["ES","NS","WS","EM","EE","LD","NEE","NWE","SEE","SWE","WAL","WM","YH"]

## 1. Load Data

To begin, data for a middle frames model must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [22]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/middle_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [23]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, REGION]

In [24]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)
training_videos = np.swapaxes(training_videos, 3, 4)

In [25]:
training_videos[:, :, :, :, 0] = (training_videos[:, :, :, :, 0] - np.mean(training_videos[:, :, :, :, 0])) / (np.std(training_videos[:, :, :, :, 0]))
training_videos[:, :, :, :, 1] = (training_videos[:, :, :, :, 1] - np.mean(training_videos[:, :, :, :, 1])) / (np.std(training_videos[:, :, :, :, 1]))

training_rainfall = (training_rainfall - np.min(training_rainfall)) / (np.max(training_rainfall) - np.min(training_rainfall))

In [26]:
print(np.min(training_rainfall, axis=0))
print(np.max(training_rainfall, axis=0))

0.0
1.0


In [27]:
def apply_augmentation(raw_rain, raw_vids):
    vids = np.copy(raw_vids)
    vids = vids + np.random.normal(0, 0.05, vids.shape)
    return np.concatenate((raw_vids, vids), axis=0), np.concatenate((raw_rain, raw_rain), axis=0)

APPLY_AUGMENTATION = True

if APPLY_AUGMENTATION:
    training_videos, training_rainfall = apply_augmentation(training_rainfall, training_videos)

In [33]:
training_videos.shape

(548, 28, 61, 121, 2)

## 2. Model Definition

Next, a CNN model architecture is defined.

In [30]:
class SlowFusion(Model):
    
    def __init__(self):
        super(SlowFusion, self).__init__()
        
        # First level
        self._first_layers = [
            self._generate_conv_layer(16, (2, 2, 2), (7, 61, 121, 2)),
            self._generate_conv_layer(16, (2, 2, 2), (7, 61, 121, 2)),
            self._generate_conv_layer(16, (2, 2, 2), (7, 61, 121, 2)),
            self._generate_conv_layer(16, (2, 2, 2), (7, 61, 121, 2))
        ]
        
        # Second level
        self._second_layers = [
            self._generate_conv_layer(16, (2, 2, 2), (6, 30, 60, 16)),
            self._generate_conv_layer(16, (2, 2, 2), (6, 30, 60, 16))
        ]
        
        # Third level
        self._third_layers = [
            self._generate_conv_layer(16, (2, 2, 2), (4, 14, 29, 16))
        ]
        

        # Final Dense layer
        self._final_layer = self._generate_dense_layer(1)
    
    def call(self, inputs):
        outputs = self._layer_one(inputs)
        #print(outputs[0].shape)
        outputs = self._layer_two(outputs)
        #print(outputs[0].shape)
        outputs = self._layer_three(outputs)
        #print(outputs.shape)
        outputs = self._final_layer(outputs)
        #print(outputs.shape)
        return outputs
        
    def _layer_one(self, inputs):
        outputs = []
        for n in range(0, 4):
            outputs.append(
                self._first_layers[n](
                    inputs[:, (n*7):((n+1)*7), :, :, :]
                )
            )
        trans_outputs = [
            tf.concat((outputs[0], outputs[1]), axis=1),
            tf.concat((outputs[2], outputs[3]), axis=1)
        ]
        return trans_outputs
    
    def _layer_two(self, inputs):
        outputs = []
        for n in range(0, 2):
            outputs.append(self._second_layers[n](inputs[n]))
        trans_outputs = [
            tf.concat((outputs[0], outputs[1]), axis=1)
        ]
        #trans_outputs = tf.concat((outputs[0], outputs[1]), axis=1)
        return trans_outputs
    
    def _layer_three(self, inputs):
        return self._third_layers[0](inputs)
    
    def _generate_conv_layer(self, filters, poolsize, input_shape):
        layer = Sequential()
        layer.add(Conv3D(
            filters, poolsize, input_shape=input_shape,
            kernel_initializer="glorot_normal"))#initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
        #layer.add(BatchNormalization())
        layer.add(Activation('relu'))
        layer.add(MaxPooling3D(pool_size=poolsize))
        return layer
    
    def _generate_dense_layer(self, output_size):
        layer = Sequential()
        layer.add(Flatten())
        layer.add(Dense(output_size, kernel_initializer="glorot_normal"))#initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
        #layer.add(BatchNormalization())
        layer.add(Activation('relu'))
        return layer

In [31]:
def model_generator(input_shape=(28, 61, 121, 2), learning_rate=0.1):
    """ This method generates a model definition. """
    model = SlowFusion()
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))#SGD(lr=learning_rate, nesterov=True))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [37]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    #mc = ModelCheckpoint(
    #    filepath=models_folder + run_name,
    #    save_best_only=True,
    #    save_weights_only=False
    #)
    history = model.fit(
        xdata,
        ydata,
        batch_size=32,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=25
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [38]:
# Flexible parameters
learning_rate = 0.01

In [ ]:
# Run each model multiple times
for i in range(0, 5):
    run_name = "SlowFusion/{}-{}".format(REGIONS[REGION], int(time.time()))#"SF_32_16_8__13__222_t-{}".format(int(time.time()))
    tb = TensorBoard(log_dir="D:/PHD_DATA/Video_11-02-2021/logs/{}".format(run_name))
    model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
    history = train_model(run_name, tb, model, training_videos, training_rainfall)
    print("Final loss: {}".format(history.history["val_loss"][-1]))

Epoch 1/25
12/12 [==============================] - 68s 5s/step - loss: 9.4742 - val_loss: 0.3546
Epoch 2/25
 7/12 [================>.............] - ETA: 19s - loss: 0.3292